In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import json
import requests
import time
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import statsmodels.stats.api as sms
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler

In [2]:
%store -r pass_list
pass_list.remove('const')

no stored variable or alias pass_list


NameError: name 'pass_list' is not defined

We will feature select and try to meet assumptions for our trained dataset before running the model again.

In [ ]:
pass_list

In [ ]:
continuous = ['date', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'yr_built', 'lat', 'long', 
              'sqft_living15', 'sqft_lot15']

In [ ]:
data = pd.read_csv("data/df_joined.csv")
df = pd.DataFrame(data)

In [ ]:
y = df.price
X = df.drop(columns = "price", axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [ ]:
X_train

In [ ]:
y_train.head()

In [ ]:
new_train = pd.concat([X_train[pass_list],y_train], axis=1) #only select features with significant p-values from pass_list

In [ ]:
new_train.head()

In [ ]:
correlation_df = new_train.corr()

In [ ]:
correlation_df.applymap(lambda val: abs(val) >= 0.75)
corr_data = df.corr().abs().stack().reset_index().sort_values(0, ascending=False)
corr_data[corr_data[0] < 1][:10]

sqft_living appears to be the addition of sqft_above and sqft_basement. Since sqft_living and sqft_above are highly correlated we will choose to drop sqft_living.  Condition 3 and Condition 4 also appear to be highly correlated so we will drop con_3. 

In [ ]:
new_train.drop(columns = ['sqft_living', 'con_3'], inplace=True)

In [ ]:
continuous.remove('sqft_living')

looking at normality of continuous variables before considering log transforming

In [ ]:
new_train[continuous].hist(figsize=[15,15]);

In [ ]:
new_train['sqft_basement'] #sqft_basement is highly zero weighted so we will drop this feature

In [ ]:
new_train.drop(columns = 'sqft_basement', inplace=True)

In [ ]:
continuous.remove('sqft_basement')

In [ ]:
new_train[new_train['sqft_lot15']==0] #sqft_lot and sqft_lot15 look zero-weighted but aren't actually.  Problem may be outliers.

In [ ]:
new_train.drop(columns = 'long', inplace=True) # removing longitude because of negative values
continuous.remove('long')

In [ ]:
log_names = [f'{column}_log' for column in new_train[continuous].columns]
df_log = np.log(new_train[continuous])
df_log.columns=log_names
df_log

In [ ]:
df_log.hist(figsize=[15,15]);

In [ ]:
def normalize(feature):
    return (feature - feature.mean()) / feature.std()
df_log_norm = df_log.apply(normalize)
df_log_norm

In [ ]:
df_log_norm.hist(figsize=[15,15]);

Now we will log and normalize our dependent variable Price

In [ ]:
y_train_log = np.log(y_train)
y_train_log

In [ ]:
y_train_log_norm = normalize(y_train_log)
y_train_log_norm

In [ ]:
#append price to continuous then drop
new_cat = new_train.drop(columns = continuous)
new_cat

In [ ]:
trained2_df = pd.concat([df_log_norm,new_cat],axis=1)

In [ ]:
trained2_df

In [ ]:
X_int = sm.add_constant(trained2_df)
model = sm.OLS(y_train ,X_int).fit()
summary = model.summary()
summary

In [ ]:
y_hat = model.predict()
train_rmse = mean_squared_error(y_train,y_hat)**.5
train_rmse